<a href="https://colab.research.google.com/github/MikeCorv/WhisperFineTuning/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes librosa ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.2 MB/s eta 0:00:00


In [ ]:
!pip install -q deepmultilingualpunctuation

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU detected. It will work, but it will be slow.")

✅ GPU Detected: Tesla T4


In [ ]:
ADAPTER_PATH = "/content/drive/MyDrive/whisper-large-v3-turbo-italian-lora"
BASE_MODEL_ID = "openai/whisper-large-v3-turbo"

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor
from peft import PeftModel

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
base_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

In [ ]:
processor = AutoProcessor.from_pretrained(ADAPTER_PATH)

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=fine_tuned_model,  # <--- UPDATED HERE
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
)

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [ ]:
#The url of the test video is available here: https://www.youtube.com/shorts/Wwz1FenTxTY (lol)

In [ ]:
#Transcription with Fine-Tuned Model
import time
from deepmultilingualpunctuation import PunctuationModel

AUDIO_FILE = "test.mp3"
start_time = time.time()

result = pipe(
    AUDIO_FILE,
    generate_kwargs={"language": "italian"}
)
end_time = time.time()

punct_model = PunctuationModel(model="oliverguhr/fullstop-punctuation-multilang-large")

print("\n" + "="*30)
print(f"⏱️ Time taken: {end_time - start_time:.2f} seconds")
print("="*30)
print("\n📝 TRANSCRIPTION:\n")
print(result['text'])
print("\n" + "="*30)
print(f"Punctuated Output: {punct_model.restore_punctuation(result['text'])}")

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0



⏱️ Time taken: 5.50 seconds

📝 TRANSCRIPTION:

 è necessario introdurre un concetto e ci stiamo lavorando di permesso di soggiorno a punti come c'è la potente a punti sono ministro dei trasporti per chi guida drogato ubriaco per chi usa il telefonino alla guida si tolgono dei punti e si sospende la patente perché la vita è sacra a maggior ragione se ti concedo la cittadinanza se ti apro le porte di casa mia e commetti reati commetti infrazioni ti tolgo quei punti e posso rispedirti al tuo paese

Punctuated Output: è necessario introdurre un concetto- e ci stiamo lavorando- di permesso di soggiorno a punti, come c'è la potente a punti, sono ministro dei trasporti. per chi guida drogato, ubriaco, per chi usa il telefonino alla guida, si tolgono dei punti e si sospende la patente. perché la vita è sacra, a maggior ragione se ti concedo la cittadinanza, se ti apro le porte di casa mia e commetti reati, commetti infrazioni, ti tolgo quei punti e posso rispedirti al tuo paese.


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(
